In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Class/Text Mining

/content/drive/MyDrive/Class/Text Mining


In [3]:
#import libraries
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.models import Sequential
import keras.utils as ku 
import pandas as pd
import numpy as np
import string, os 
import warnings
import jieba
import random
import sys
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
#import dataset
jay = pd.read_csv("data2/JayChou2_new_2.csv")
piggy = pd.read_csv("data2/PiggyLo2_new_2.csv")
jolin = pd.read_csv("data2/Jolin2_new_2.csv")
all = pd.read_csv("data2/AllMalePrizedArtist_2.csv")

#choose
df = all
lyrics = df['lyric_full']

lyrics = lyrics.values.tolist()

lyrics_full = []

# for song in lyrics:
#     song = jieba.lcut(song)
#     lyrics_full.append(song)


In [6]:
lyrics_full = df['lyric_ckip']

In [7]:
#split in to segment
seg_len = 5
stride = 2

segments = []
next_chars = []
for i in range(len(lyrics_full)):
    for j in range(0, len(lyrics_full[i]) - seg_len, stride):
        segments.append(lyrics_full[i][j: j + seg_len])
        next_chars.append(lyrics_full[i][j + seg_len])

In [ ]:
#construct one-hot vectors
vocab = []
for i in range(len(lyrics_full)):
    tokens = sorted(lyrics_full[i])
    for token in tokens:
        if token not in vocab:
            vocab.append(token)
vocab_size = len(vocab)

segments_vector = np.zeros((len(segments), seg_len, vocab_size), int)
next_chars_vector = np.zeros((len(segments), vocab_size), int)
for i, segment in enumerate(segments):
    for j, char in enumerate(segment):
        segments_vector[i, j, vocab.index(char)] = 1
    next_chars_vector[i, vocab.index(next_chars[i])] = 1


In [ ]:
#LSTM model
model = Sequential()
model.add(LSTM(16, input_shape=(seg_len, vocab_size)))
model.add(Dense(vocab_size, activation='softmax'))
model.summary()


In [ ]:
#Train model
from keras.optimizers import RMSprop
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.fit(segments_vector, next_chars_vector, batch_size=512, epochs=20)

In [ ]:
def sample(pred, temperature):
    pred = pred ** (1 / temperature)
    pred = pred / np.sum(pred)
    return np.argmax(pred)

In [ ]:
text_len = 60
temperature = 0.2

start_index_song = random.randint(0, len(lyrics_full) - 1)
start_index_lyrics = random.randint(0, len(lyrics_full[start_index_song]) - seg_len - 1)
generated_text_list = lyrics_full[start_index_song][start_index_lyrics:start_index_lyrics + seg_len]
generated_text = ""

for i in range(len(generated_text_list)):
    generated_text += str(generated_text_list[i])
sys.stdout.write(generated_text)

for i in range(text_len):
    sampled = np.zeros((1, seg_len, vocab_size), int)
    for j, char in enumerate(generated_text_list):
        sampled[0, j, vocab.index(char)] = 1
    pred = model.predict(sampled, verbose=0)[0]
    next_index = sample(pred, temperature)
    next_char = vocab[next_index]
    
    generated_text_list.append(next_char)
    generated_text_list = generated_text_list[1:]
    
    sys.stdout.write(str(next_char))



她再等，沒差，再再再蕭邦，我你說你的話，不要再這樣打我媽媽，我了了了，妳的。我卻沒有多，卻忘有較我我的可。沒有不能再愛再你再